In [1]:
!pip install praw
!pip install psutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.4 MB/s eta 0:00:00


## Imports

In [2]:
import praw
import os
import requests
import pandas as pd
import time
import json
from io import BytesIO
from PIL import Image
from pathlib import Path
from praw.models import MoreComments
from huggingface_hub import HfApi, HfFolder
from datasets import Dataset, Features, Value, Array3D, Sequence
from datasets import Image as DsImage
import numpy as np
from tqdm.notebook import tqdm
import psutil

## Configuration

In [ ]:
REDDIT_CLIENT_ID = ""
REDDIT_CLIENT_SECRET = ""
REDDIT_USER_AGENT = "reddit-roomstyle-scraper:v1.0 (by /u/pawmyer)"
SUBREDDIT = "malelivingspace"
POST_LIMIT = 10_000
SLEEP = 1.2
CHECKPOINT_EVERY = 100
SAVE_PATH = "./temp/reddit_malelivingspace_scraped.parquet"
HF_REPO_ID = "partzel/reddit-malelivingspace-scraped"
HF_TOKEN = ""

In [4]:
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

In [5]:
Path("temp").mkdir(exist_ok=True)
seen_ids = set()
data = []

## Scraping

Checkpointing

In [6]:
MAX_SIZE = (512, 512)

def fetch_image_bitmap(url):
    try:
        resp = requests.get(url, stream=True, timeout=5)
        resp.raise_for_status()
        img = Image.open(BytesIO(resp.content)).convert("RGB")
        img.thumbnail(MAX_SIZE)
        return img
    except Exception as e:
        print(f"[IMAGE ERROR] {url}: {e}")
        return None

In [ ]:
counter = 0

batch_size = 20
batch_id = 0
batch = []
for submission in tqdm(reddit.subreddit(SUBREDDIT).new(limit=POST_LIMIT)):
    # Monitor RAM usage for Kernel problems
    print(f"RAM used: {psutil.virtual_memory().percent}%")
    
    time.sleep(SLEEP)
    if submission.id in seen_ids:
        continue

    # Image URLs
    image_urls = []
    if submission.url.endswith(('.jpg', '.jpeg', '.png')):
        image_urls = [submission.url]
    elif getattr(submission, 'is_gallery', False):
        try:
            for item in submission.media_metadata.values():
                url = item['s']['u'].replace("&amp;", "&")
                image_urls.append(url)
        except Exception:
            pass

    if not image_urls:
        continue

    # Download images
    images = [fetch_image_bitmap(url) for url in image_urls]
    images = [np.array(img).tolist() for img in images if img is not None]

    if not images:
        continue

    # Get top comments
    try:
        submission.comments.replace_more(limit=0)
        top_comments = [c.body for c in submission.comments[:5] if isinstance(c, praw.models.Comment)]
    except Exception as e:
        print(f"[COMMENT FAIL] {submission.id}: {e}")
        top_comments = []

    batch.append({
        "post_id": submission.id,
        "title": submission.title,
        "top_comments": top_comments,
        "images": images  # list of RGB arrays
    })

    seen_ids.add(submission.id)
    counter += 1

    # Checkpoint to Parquet
    features = Features({
        "post_id": Value("string"),
        "title": Value("string"),
        "top_comments": Sequence(Value("string")),
        "images": Sequence(DsImage())
    })
    
    if len(batch) == batch_size:
        ds = Dataset.from_list(batch, features=features)
        
        # Push as a new split or with a custom config
        ds.push_to_hub(HF_REPO_ID, token=HF_TOKEN, split=f"batch_{batch_id}")
        print(f"Pushed batch {batch_id}")
        
        batch.clear()
        batch_id += 1


0it [00:00, ?it/s]

RAM used: 3.4%
RAM used: 3.4%
RAM used: 3.7%
RAM used: 3.9%
RAM used: 3.9%
RAM used: 4.0%
RAM used: 4.1%
RAM used: 4.3%


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


RAM used: 4.4%
RAM used: 5.1%
RAM used: 5.5%
RAM used: 5.7%
RAM used: 5.8%
RAM used: 5.9%
RAM used: 5.9%
RAM used: 6.0%
RAM used: 6.8%
RAM used: 6.9%
RAM used: 7.8%
RAM used: 7.9%


/usr/local/lib/python3.11/dist-packages/datasets/features/image.py:338: UserWarning: Downcasting array dtype int64 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

Pushed batch 42
RAM used: 7.9%
RAM used: 7.6%
